# LangGraph with Structured Output and Conditional Routing

This notebook demonstrates advanced LangGraph concepts:
- **Structured Output**: Using Pydantic models for type-safe LLM responses
- **Conditional Routing**: Dynamic graph paths based on user intent
- **Multiple Nodes**: Coordinating different processing functions
- **LLM Integration**: Working with AWS Bedrock and Claude
- **Decision Making**: Using LLMs to route between different workflows

## Use Case: Intelligent Librarian System

We'll build a system that can:
1. **Identify Author**: Given a book name, find the author and provide author persona
2. **Suggest Books**: Recommend similar books based on user preferences
3. **Smart Routing**: Automatically determine which service the user needs

This pattern is useful for:
- **Multi-service Applications**: When you have different capabilities
- **Intent Classification**: Routing users to appropriate handlers
- **Structured Data Extraction**: Getting consistent, typed responses from LLMs

## 1. Essential Imports and Setup

Let's import all the necessary libraries for our advanced LangGraph application:

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image
from typing import TypedDict
from dotenv import load_dotenv
import boto3
import os

## 2. State Definition

Our state needs to handle multiple types of data:
- **messages**: For conversation flow
- **book**: Book information
- **author_name**: Author identification
- **author_persona**: Author's personality/greeting
- **suggestions**: Book recommendations

### Design Considerations:
- **Flexibility**: State should accommodate different workflows
- **Type Safety**: Clear field definitions for better debugging
- **Extensibility**: Easy to add new fields as features grow

In [ ]:
from langgraph.graph import MessagesState

class BookState(MessagesState):
    """Enhanced state for our librarian system.
    
    Inherits from MessagesState to get message handling capabilities,
    and adds book-specific fields for our use case.
    """
    book: str = ""                    # Book title or query
    author_name: str = ""             # Identified author
    author_persona: str = ""          # Author's greeting/introduction
    suggestions: list = []            # List of book suggestions

## 3. LLM Setup with AWS Bedrock

We'll use AWS Bedrock with Claude for our LLM capabilities.

### Why AWS Bedrock?
- **Enterprise Ready**: Built for production use
- **Multiple Models**: Access to various LLMs
- **Scalability**: Handles high-volume requests
- **Security**: Enterprise-grade security features

### Alternative LLM Providers:
- **OpenAI**: `ChatOpenAI` for GPT models
- **Anthropic**: Direct `ChatAnthropic` integration
- **Local Models**: `Ollama` for on-premise deployment
- **Azure**: `AzureChatOpenAI` for Microsoft cloud

In [ ]:
from langchain_aws import ChatBedrock

def get_chat_model():
    """Initialize AWS Bedrock ChatBedrock model.
    
    This function sets up our connection to Claude via AWS Bedrock.
    Make sure you have AWS credentials configured.
    
    Returns:
        ChatBedrock: Configured LLM instance
    """
    load_dotenv()
    
    # Create Bedrock client with credentials
    bedrock_client = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
    )
    
    # Initialize ChatBedrock with Claude Sonnet
    return ChatBedrock(
        client=bedrock_client,
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    )

## 4. Structured Output with Pydantic

**Structured Output** ensures LLMs return data in a consistent, typed format.

### Benefits of Structured Output:
- **Type Safety**: Guaranteed data structure
- **Validation**: Automatic data validation
- **Consistency**: Same format every time
- **Integration**: Easy to use in downstream processing

### Methods for Structured Output:
1. **with_structured_output()**: Cleanest approach (recommended)
2. **bind_tools()**: More control over tool calling
3. **Output Parsers**: Manual parsing (less reliable)

We'll use `with_structured_output()` as it's the most reliable method.

In [ ]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Model for author identification responses
class AnswerQuestion(BaseModel):
    """Structured response for author identification queries."""
    author_name: str = Field(description="The author of the mentioned book")
    author_persona: str = Field(
        description="A brief greeting/introduction from the author of the mentioned book"
    )

# Model for book suggestion responses
class SuggestionAnswer(BaseModel):
    """Structured response for book suggestion queries."""
    books_titles: list = Field(
        description="A list of 3 books to be suggested based on user request"
    )

# Model for decision making
class Decision(BaseModel):
    """Decision model for routing user queries."""
    author: bool = Field(
        description="True if the user query is about finding an author of a book"
    )
    suggestion: bool = Field(
        description="True if the user query is asking for book suggestions"
    )

## 5. Prompt Engineering

Well-crafted prompts are crucial for reliable LLM behavior.

### Prompt Design Principles:
- **Clear Instructions**: Specific about what you want
- **Context Setting**: Establish the LLM's role
- **Output Format**: Specify expected response structure
- **Examples**: Show desired behavior when needed

In [ ]:
# Prompt for author identification
AUTHOR_PROMPT = """You are a knowledgeable librarian with access to vast literary knowledge.

Your tasks:
1. Identify the author of the book mentioned by the user
2. Provide a brief, engaging greeting as if you were that author

The greeting should:
- Be written in first person as the author
- Reflect the author's personality and writing style
- Be warm and welcoming to the reader
- Reference their notable works when appropriate
"""

# Prompt for book suggestions
SUGGESTION_PROMPT = """You are a librarian who suggests books based on user requests.

Your task:
- Analyze the user's request for book recommendations
- Suggest exactly 3 books that match their interests
- Consider genre, themes, writing style, and popularity
- Provide diverse recommendations when possible

Focus on quality recommendations that truly match the user's expressed interests.
"""

# Prompt for decision making
DECISION_PROMPT = """Analyze the user's query and determine their intent.

Classification rules:
- Set 'author' to True if they're asking about who wrote a specific book
- Set 'suggestion' to True if they want book recommendations or suggestions
- Only one should be True based on the primary intent
- If unclear, lean towards the most likely interpretation

Examples:
- "Who wrote 1984?" → author: True, suggestion: False
- "Suggest books like Harry Potter" → author: False, suggestion: True
- "I need book recommendations" → author: False, suggestion: True
"""

## 6. Building LangChain Chains

**Chains** combine prompts with LLMs to create reusable processing units.

### Chain Components:
1. **Prompt Template**: Structures the input
2. **LLM**: Processes the prompt
3. **Output Parser**: Formats the response (handled by structured output)

### Why Use Chains?
- **Reusability**: Same logic across different contexts
- **Modularity**: Easy to test and modify individual components
- **Consistency**: Standardized processing patterns

In [ ]:
def author_chain():
    """Create a chain for author identification with structured output.
    
    This chain:
    1. Takes user messages about books
    2. Identifies the author
    3. Creates an author persona greeting
    4. Returns structured AnswerQuestion object
    
    Returns:
        Chain that outputs AnswerQuestion objects
    """
    llm = get_chat_model()
    
    # Use structured output for reliable data format
    structured_llm = llm.with_structured_output(AnswerQuestion)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", AUTHOR_PROMPT),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    return prompt | structured_llm

def suggestion_chain():
    """Create a chain for book suggestions with structured output.
    
    This chain:
    1. Analyzes user's book preferences
    2. Generates 3 relevant book suggestions
    3. Returns structured SuggestionAnswer object
    
    Returns:
        Chain that outputs SuggestionAnswer objects
    """
    llm = get_chat_model()
    structured_llm = llm.with_structured_output(SuggestionAnswer)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", SUGGESTION_PROMPT),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    return prompt | structured_llm

def decision_chain():
    """Create a chain for intent classification.
    
    This chain determines whether the user wants:
    - Author information (author=True)
    - Book suggestions (suggestion=True)
    
    Returns:
        Chain that outputs Decision objects
    """
    llm = get_chat_model()
    structured_llm = llm.with_structured_output(Decision)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", DECISION_PROMPT),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    return prompt | structured_llm

## 7. Testing Individual Chains

Before building the full graph, let's test our chains individually to ensure they work correctly:

In [ ]:
from langchain.schema import HumanMessage

# Test the author chain
print("Testing Author Chain:")
print("=" * 50)

author_test_chain = author_chain()
author_result = author_test_chain.invoke({
    "messages": [HumanMessage(content="Who is the author of Khushwantnama?")]
})

print(f"Author: {author_result.author_name}")
print(f"Persona: {author_result.author_persona}")
print(f"Type: {type(author_result)}")

print("\n" + "=" * 50)
print("Testing Suggestion Chain:")
print("=" * 50)

suggestion_test_chain = suggestion_chain()
suggestion_result = suggestion_test_chain.invoke({
    "messages": [HumanMessage(content="Suggest some books like Macbeth")]
})

print(f"Suggestions: {suggestion_result.books_titles}")
print(f"Type: {type(suggestion_result)}")

## 8. Creating Graph Nodes

**Nodes** are the processing units in our graph. Each node:
1. Receives the current state
2. Processes it using our chains
3. Returns state updates

### Critical Node Requirements:
- **State Updates**: Must return dictionaries that update the state
- **Error Handling**: Should handle chain failures gracefully
- **Type Consistency**: Ensure returned data matches state expectations

### Common Node Patterns:
- **Processing Nodes**: Transform data using LLMs
- **Decision Nodes**: Route based on conditions
- **Integration Nodes**: Call external APIs or services

In [ ]:
def author_node(state: BookState) -> BookState:
    """Process author identification requests.
    
    This node:
    1. Uses the author chain to identify the book's author
    2. Extracts structured data from the response
    3. Updates the state with author information
    
    Args:
        state: Current state with user messages
        
    Returns:
        State update with author_name and author_persona
    """
    print("📚 Processing author identification request...")
    
    chain = author_chain()
    result = chain.invoke({"messages": state["messages"]})
    
    print(f"✅ Found author: {result.author_name}")
    
    # Return state updates - this is crucial!
    return {
        "author_name": result.author_name,
        "author_persona": result.author_persona
    }

def suggestion_node(state: BookState) -> BookState:
    """Process book suggestion requests.
    
    This node:
    1. Uses the suggestion chain to generate book recommendations
    2. Extracts the list of suggested books
    3. Updates the state with suggestions
    
    Args:
        state: Current state with user messages
        
    Returns:
        State update with book suggestions
    """
    print("📖 Processing book suggestion request...")
    
    chain = suggestion_chain()
    result = chain.invoke({"messages": state["messages"]})
    
    print(f"✅ Generated {len(result.books_titles)} suggestions")
    
    # Return state updates
    return {
        "suggestions": result.books_titles
    }

## 9. Conditional Routing Function

This function determines which path the graph should take based on user intent.

### Routing Strategies:
1. **LLM-Based**: Use AI to classify intent (our approach)
2. **Rule-Based**: Use keywords or patterns
3. **Hybrid**: Combine both approaches
4. **ML Classification**: Train a dedicated classifier

### Why LLM-Based Routing?
- **Flexibility**: Handles natural language variations
- **Context Awareness**: Understands nuanced requests
- **No Training Required**: Works out of the box
- **Easy Updates**: Modify behavior through prompt changes

In [ ]:
def condition(state: BookState) -> str:
    """Determine routing based on user intent.
    
    This function:
    1. Analyzes user messages using the decision chain
    2. Classifies intent as author lookup or book suggestions
    3. Returns routing decision for the graph
    
    Args:
        state: Current state with user messages
        
    Returns:
        "author" for author queries, "suggestion" for book recommendations
    """
    print("🤔 Analyzing user intent...")
    
    chain = decision_chain()
    decision = chain.invoke({"messages": state["messages"]})
    
    if decision.author:
        print("➡️  Routing to author identification")
        return "author"
    elif decision.suggestion:
        print("➡️  Routing to book suggestions")
        return "suggestion"
    else:
        print("⚠️  Unclear intent, defaulting to suggestions")
        return "suggestion"

## 10. Building the Complete Graph

Now we'll assemble all components into a working graph.

### Graph Architecture:
```
START → [condition] → author_node → END
                   → suggestion_node → END
```

### Key Design Decisions:
- **Single Entry Point**: All requests start at the same place
- **Dynamic Routing**: LLM determines the appropriate path
- **Parallel Paths**: Different nodes for different capabilities
- **Clean Termination**: Both paths lead to END

In [ ]:
# Build the graph
graph = StateGraph(BookState)

# Add our processing nodes
graph.add_node("author_node", author_node)
graph.add_node("suggestion_node", suggestion_node)

# Add conditional routing from START
graph.add_conditional_edges(
    START,           # From the start
    condition,       # Use this function to decide
    {
        "author": "author_node",           # Route to author identification
        "suggestion": "suggestion_node"   # Route to book suggestions
    }
)

# Both paths end the graph
graph.add_edge("author_node", END)
graph.add_edge("suggestion_node", END)

# Compile the graph
app = graph.compile()

print("✅ Graph compiled successfully!")

## 11. Graph Visualization

Let's visualize our graph structure to understand the flow:

In [ ]:
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization error: {e}")
    print("\nGraph Structure:")
    print("START → [condition function]")
    print("  ├─ 'author' → author_node → END")
    print("  └─ 'suggestion' → suggestion_node → END")

## 12. Testing the Complete System

Let's test both paths of our graph with different types of queries:

In [ ]:
# Test 1: Author identification
print("🧪 Test 1: Author Identification")
print("=" * 50)

author_query = {
    "messages": [HumanMessage(content="Who is the author of Khushwantnama?")]
}

result1 = app.invoke(author_query)

print(f"\n📊 Results:")
print(f"Author: {result1.get('author_name', 'Not found')}")
print(f"Persona: {result1.get('author_persona', 'Not available')}")
print(f"Suggestions: {result1.get('suggestions', 'None')}")

In [ ]:
# Test 2: Book suggestions
print("\n🧪 Test 2: Book Suggestions")
print("=" * 50)

suggestion_query = {
    "messages": [HumanMessage(content="Suggest some books like Macbeth")]
}

result2 = app.invoke(suggestion_query)

print(f"\n📊 Results:")
print(f"Author: {result2.get('author_name', 'Not found')}")
print(f"Persona: {result2.get('author_persona', 'Not available')}")
print(f"Suggestions: {result2.get('suggestions', 'None')}")

In [ ]:
# Test 3: Edge case - ambiguous query
print("\n🧪 Test 3: Ambiguous Query")
print("=" * 50)

ambiguous_query = {
    "messages": [HumanMessage(content="Tell me about books")]
}

result3 = app.invoke(ambiguous_query)

print(f"\n📊 Results:")
print(f"Author: {result3.get('author_name', 'Not found')}")
print(f"Persona: {result3.get('author_persona', 'Not available')}")
print(f"Suggestions: {result3.get('suggestions', 'None')}")

## Key Takeaways

### What We Learned:
1. **Structured Output**: Using Pydantic models for reliable LLM responses
2. **Conditional Routing**: Dynamic graph paths based on user intent
3. **Multi-Node Coordination**: Building complex workflows with multiple capabilities
4. **LLM Integration**: Working with AWS Bedrock and Claude effectively
5. **Intent Classification**: Using LLMs to route between different services
6. **State Management**: Handling complex state with multiple data types

### When to Use This Pattern:
- **Multi-Service Applications**: When you have different capabilities to offer
- **Intent-Based Routing**: When user requests need different processing
- **Structured Data Extraction**: When you need consistent, typed responses
- **Conversational AI**: When building chatbots with multiple skills

### Best Practices Demonstrated:
- **Modular Design**: Separate chains for different capabilities
- **Type Safety**: Using Pydantic for structured responses
- **Clear Separation**: Distinct nodes for different processing types
- **Robust Routing**: LLM-based intent classification
- **Error Handling**: Graceful fallbacks for unclear intents

### Common Pitfalls to Avoid:
- **Wrong Return Format**: Nodes must return state update dictionaries
- **Missing State Fields**: Ensure state includes all needed fields
- **Poor Prompts**: Unclear prompts lead to inconsistent routing
- **No Fallbacks**: Always handle edge cases in routing logic

### Next Steps:
- **Notebook 3**: Build simple chatbots with LangGraph
- **Notebook 4**: Add memory and persistence to conversations
- **Notebook 5**: Implement SQLite-based persistent memory

### Extensions You Could Add:
- **More Services**: Add genre classification, book reviews, etc.
- **Better Routing**: Multi-level intent classification
- **Error Recovery**: Retry logic for failed LLM calls
- **Caching**: Store frequent queries for faster responses
- **User Profiles**: Personalized recommendations based on history